# SEIRX model runs for prevention measures investigation in schools

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
from os.path import join
import os
import shutil
import json

from scseirx.model_school import SEIRX_school
from scseirx import analysis_functions as af
from scseirx import construct_school_network as csn

import data_creation_functions as dcf

# for progress bars
from ipywidgets import IntProgress
from IPython.display import display
import time

## Simulation settings

In [ ]:
# baseic prevention measures in place. Additional measures will be added
# throughout the measure screening simulations
with open('params/intervention_screening_measures.json', 'r') as fp:
    measures = json.load(fp)
# simulation parameters, specifically the
# - base transmission risk (calibrated from household transmissions)
# - subclinical transmission modifier (literature value)
# - exposure duration, time until symtpoms and infection duration (lit. values)
# - age symptom discount (fit to empirical observations)
# - age transmission discount (from the calibration)
# - weights of transmission risks for intermediate & far contacts (calibration)
with open('params/intervention_screening_simulation_parameters.json', 'r') as fp:
    simulation_params = json.load(fp)
# characteristics (# classes, # students / class, # teachers) of the "average" 
# school, depending on school type. These characteristics were determined in 
# interviews with Austrian teachers and from statistics about Austrian schools 
# (year 2017/18, page 10: https://www.bmbwf.gv.at/Themen/schule/schulsystem/gd.html)
# NOTE: "students" indicates the number of students per class
with open('params/intervention_screening_school_characteristics.json', 'r') as fp:
    school_characteristics = json.load(fp)

## Screening parameters

In [ ]:
## parameter ranges 
# specifies whether the index case will be introduced via an
# employee or a resident
index_cases = ['student', 'teacher']
# test technologies (and test result turnover times) used in the
# different scenarios
test_types = ['same_day_antigen']
# student and teacher streening intervals (in days)
s_screen_range = [None, 3, 7]
t_screen_range = [None, 3, 7]
# specifies whether teachers wear masks
student_masks = [True, False]
teacher_masks = [True, False]
half_classes = [True, False]
# specifies whether there is ventilation or not
transmission_risk_ventilation_modifiers = [1, 0.36]

params = [(i, j, k, l, m, n, o, p)\
              for i in index_cases \
              for j in test_types \
              for k in s_screen_range \
              for l in t_screen_range \
              for m in student_masks \
              for n in teacher_masks \
              for o in half_classes \
              for p in transmission_risk_ventilation_modifiers]

N_configs = len(params)
print('there are {} different parameter combinations'.format(N_configs))

## Run simulations

In [ ]:
# paths for data I/O
contact_network_src = '../../data/contact_networks/representative_schools'
dst = '../../data/intervention_measures/simulation_results'

In [ ]:
school_types = ['primary', 'primary_dc', 'lower_secondary',
                'lower_secondary_dc', 'upper_secondary', 'secondary']

# number of runs per ensemble
# Note: this is set to 1 for testing purposes. To get properly converged 
# statistics, this has to be >= 500. Running such a high number of simulations
# will take a long time if run on just a single core. Therefore I strongly
# recommend to run the below code on several cores at once. It is easy to
# parallelize as each ensemble can be run on a different core and there are
# no interdependencies between ensembles. Results can be collected afterwards
# and evaulated together.
N_runs = 1

# progress bar
f = IntProgress(min=0, max=len(school_types) * len(params)) 
display(f)
c = 0

results = pd.DataFrame()
for school_type in school_types:
    try:
        os.mkdir(join(dst, school_type))
    except FileExistsError:
        pass

    for p in params:
        # get the parameter values
        index_case, ttype, s_screen_interval, t_screen_interval, student_mask, \
                teacher_mask, half_classes, ventilation_mod = p
        
        # run the ensemble with the given parameter combination and school type
        ensemble_results = dcf.run_ensemble(N_runs, school_type, measures,\
                simulation_params, school_characteristics, contact_network_src,\
                dst, index_case, ttype, s_screen_interval, t_screen_interval,\
                student_mask, teacher_mask, half_classes, ventilation_mod)


## Aggregate results

In [ ]:
src_path = '../../data/intervention_measures/simulation_results'
for stype in ['primary', 'primary_dc', 'lower_secondary','lower_secondary_dc', 
              'upper_secondary', 'secondary']:
    print(stype)
    data = dcf.get_data(stype, src_path)
    data['measure'] = np.nan
    dcf.set_individual_measures(data)
    dcf.set_measure_packages(data)
    data.to_csv(join(src_path, '{}_combined_ensembles.csv'.format(stype)), index=False)